# Training a Simple CNN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from data.image_dataset import ImageDataset
from models.cnn import CNN
from models.nvgaze import NVGaze
from training.train import *
from training.eval import *
from data.convert_labels import *
import matplotlib.pyplot as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
train_dir_path = 'data/real/01_20_160_120/train'
train_annotations_file_path = f'{train_dir_path}/01_20_160_120_train.csv'
test_dir_path = 'data/real/01_20_160_120/test'
test_annotations_file_path = f'{test_dir_path}/01_20_160_120_test.csv'

# Can override arguments specifying class boundaries
target_transform = lambda target: convert_labels(target)

train_val_dataset = ImageDataset(
  annotations_file=train_annotations_file_path,
  img_dir=train_dir_path,
  transform=None,
  target_transform=target_transform
)
test_dataset = ImageDataset(
  annotations_file=test_annotations_file_path,
  img_dir=test_dir_path,
  transform=None,
  target_transform=target_transform
)

generator = torch.Generator().manual_seed(0)
train_dataset, val_dataset = random_split(train_val_dataset, [0.9, 0.1], generator=generator)

In [ ]:
# Create Dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_loader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# print(np.unique(train_labels))

In [ ]:
# model = CNN(input_dims=(160, 120))
model = NVGaze(input_dims=(160, 120))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

results = train(model, criterion, optimizer, train_loader, val_loader, epochs=30)

In [ ]:
loss_history, train_acc_history, val_acc_history = results
plt.subplot(2, 1, 1)
plt.plot(loss_history, 'o')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(train_acc_history, '-o')
plt.plot(val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
test_acc = evaluate_acc(model, test_loader)
print(test_acc)